<a href="https://colab.research.google.com/github/ralfcam/LANL-Earthquake-Prediction/blob/master/TFRESH_for_LANL_Earthquake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TFRESH for LANL Earthquake

## Imports

In [0]:
!pip install tsfresh

# ERROR: google-colab 1.0.0 has requirement pandas~=0.24.0, but you'll have pandas 0.23.4 which is incompatible.
# Installing collected packages: pandas, tsfresh
#   Found existing installation: pandas 0.24.2
#     Uninstalling pandas-0.24.2:
#       Successfully uninstalled pandas-0.24.2
# Successfully installed pandas-0.23.4 tsfresh-0.11.2
# WARNING: The following packages were previously imported in this runtime:
#   [pandas]
# You must restart the runtime in order to use newly installed versions.

In [0]:
import matplotlib.pyplot as plt
import lightgbm as lgb
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
import numpy as np
import pandas as pd
from tsfresh.feature_extraction import MinimalFCParameters, EfficientFCParameters ,ComprehensiveFCParameters
from sklearn.model_selection import KFold, cross_validate, cross_val_predict
import timeit
import seaborn as sns
import sys
import time
from tqdm import tqdm, tqdm_notebook
import warnings

import multiprocessing as mp

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.kernel_ridge import KernelRidge

from sklearn.preprocessing import MinMaxScaler, StandardScaler,QuantileTransformer

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from matplotlib import pyplot

from google.colab import files, drive

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
warnings.filterwarnings("ignore")
np.seterr(divide='ignore', invalid='ignore')
np.seterr(divide='ignore')
# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

{'divide': 'ignore', 'invalid': 'ignore', 'over': 'warn', 'under': 'ignore'}

## Kaggle Sync.

In [0]:
!pip install kaggle
!mkdir .kaggle

In [0]:
import json
import os

token = {"username":"","key":""}

os.environ['KAGGLE_CONFIG_DIR'] = "/content/.kaggle"

with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

!kaggle config set -n path -v{/content}
!chmod 600 /content/.kaggle/kaggle.json

# !kaggle datasets list

- path is now set to: {/content}


In [0]:
!kaggle competitions download -c LANL-Earthquake-Prediction -p /content

  0% 0.00/33.3k [00:00<?, ?B/s]
100% 33.3k/33.3k [00:00<00:00, 21.7MB/s]
 96% 232M/242M [00:01<00:00, 149MB/s]
100% 242M/242M [00:01<00:00, 142MB/s]
100% 2.02G/2.03G [00:16<00:00, 113MB/s]
100% 2.03G/2.03G [00:16<00:00, 129MB/s]


In [0]:
%%capture unzip_hist
!unzip \*.zip

## Google Drive Sync.

In [0]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### Move Data to Drive

In [0]:
# !mkdir /content/gdrive/My\ Drive/LANL_Earthquake_Data/data_csv
!mv *.csv /content/gdrive/My\ Drive/LANL_Earthquake_Data/data_csv 

In [0]:
!rm -rf /content/*.zip

## Read Data

In [0]:
%%time
# train_path = '/content/gdrive/My Drive/LANL_Earthquake_Data/data_csv/train.csv'
train_path = '/content/train.csv'

train = pd.read_csv(train_path, dtype={'acoustic_data': np.int16,
                                       'time_to_failure': np.float32})

CPU times: user 2min 9s, sys: 44.9 s, total: 2min 54s
Wall time: 3min 3s


## Features dictionaries

### All Features

In [0]:
settings={'variance_larger_than_standard_deviation': None,
 'has_duplicate_max': None,
 'has_duplicate_min': None,
 'has_duplicate': None,
 'sum_values': None,
 'abs_energy': None,
 'mean_abs_change': None,
 'mean_change': None,
 'mean_second_derivative_central': None,
 'median': None,
 'mean': None,
 'length': None,
 'standard_deviation': None,
 'variance': None,
 'skewness': None,
 'kurtosis': None,
 'absolute_sum_of_changes': None,
 'longest_strike_below_mean': None,
 'longest_strike_above_mean': None,
 'count_above_mean': None,
 'count_below_mean': None,
 'last_location_of_maximum': None,
 'first_location_of_maximum': None,
 'last_location_of_minimum': None,
 'first_location_of_minimum': None,
 'percentage_of_reoccurring_datapoints_to_all_datapoints': None,
 'percentage_of_reoccurring_values_to_all_values': None,
 'sum_of_reoccurring_values': None,
 'sum_of_reoccurring_data_points': None,
 'ratio_value_number_to_time_series_length': None,
 'sample_entropy': None,
 'maximum': None,
 'minimum': None,
 'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
 'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
 'cid_ce': [{'normalize': True}, {'normalize': False}],
 'symmetry_looking': [{'r': 0.0},
  {'r': 0.05},
  {'r': 0.1},
  {'r': 0.15000000000000002},
  {'r': 0.2},
  {'r': 0.25},
  {'r': 0.30000000000000004},
  {'r': 0.35000000000000003},
  {'r': 0.4},
  {'r': 0.45},
  {'r': 0.5},
  {'r': 0.55},
  {'r': 0.6000000000000001},
  {'r': 0.65},
  {'r': 0.7000000000000001},
  {'r': 0.75},
  {'r': 0.8},
  {'r': 0.8500000000000001},
  {'r': 0.9},
  {'r': 0.9500000000000001}],
 'large_standard_deviation': [{'r': 0.05},
  {'r': 0.1},
  {'r': 0.15000000000000002},
  {'r': 0.2},
  {'r': 0.25},
  {'r': 0.30000000000000004},
  {'r': 0.35000000000000003},
  {'r': 0.4},
  {'r': 0.45},
  {'r': 0.5},
  {'r': 0.55},
  {'r': 0.6000000000000001},
  {'r': 0.65},
  {'r': 0.7000000000000001},
  {'r': 0.75},
  {'r': 0.8},
  {'r': 0.8500000000000001},
  {'r': 0.9},
  {'r': 0.9500000000000001}],
 'quantile': [{'q': 0.1},
  {'q': 0.2},
  {'q': 0.3},
  {'q': 0.4},
  {'q': 0.6},
  {'q': 0.7},
  {'q': 0.8},
  {'q': 0.9}],
 'autocorrelation': [{'lag': 0},
  {'lag': 1},
  {'lag': 2},
  {'lag': 3},
  {'lag': 4},
  {'lag': 5},
  {'lag': 6},
  {'lag': 7},
  {'lag': 8},
  {'lag': 9}],
 'agg_autocorrelation': [{'f_agg': 'mean', 'maxlag': 40},
  {'f_agg': 'median', 'maxlag': 40},
  {'f_agg': 'var', 'maxlag': 40}],
 'partial_autocorrelation': [{'lag': 0},
  {'lag': 1},
  {'lag': 2},
  {'lag': 3},
  {'lag': 4},
  {'lag': 5},
  {'lag': 6},
  {'lag': 7},
  {'lag': 8},
  {'lag': 9}],
 'number_cwt_peaks': [{'n': 1}, {'n': 5}],
 'number_peaks': [{'n': 1}, {'n': 3}, {'n': 5}, {'n': 10}, {'n': 50}],
 'binned_entropy': [{'max_bins': 10}],
 'index_mass_quantile': [{'q': 0.1},
  {'q': 0.2},
  {'q': 0.3},
  {'q': 0.4},
  {'q': 0.6},
  {'q': 0.7},
  {'q': 0.8},
  {'q': 0.9}],
 'cwt_coefficients': [{'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 2},
  {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 5},
  {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 10},
  {'widths': (2, 5, 10, 20), 'coeff': 0, 'w': 20},
  {'widths': (2, 5, 10, 20), 'coeff': 1, 'w': 2},
  {'widths': (2, 5, 10, 20), 'coeff': 1, 'w': 5},
  {'widths': (2, 5, 10, 20), 'coeff': 1, 'w': 10},
  {'widths': (2, 5, 10, 20), 'coeff': 1, 'w': 20},
  {'widths': (2, 5, 10, 20), 'coeff': 2, 'w': 2},
  {'widths': (2, 5, 10, 20), 'coeff': 2, 'w': 5},
  {'widths': (2, 5, 10, 20), 'coeff': 2, 'w': 10},
  {'widths': (2, 5, 10, 20), 'coeff': 2, 'w': 20},
  {'widths': (2, 5, 10, 20), 'coeff': 3, 'w': 2},
  {'widths': (2, 5, 10, 20), 'coeff': 3, 'w': 5},
  {'widths': (2, 5, 10, 20), 'coeff': 3, 'w': 10},
  {'widths': (2, 5, 10, 20), 'coeff': 3, 'w': 20},
  {'widths': (2, 5, 10, 20), 'coeff': 4, 'w': 2},
  {'widths': (2, 5, 10, 20), 'coeff': 4, 'w': 5},
  {'widths': (2, 5, 10, 20), 'coeff': 4, 'w': 10},
  {'widths': (2, 5, 10, 20), 'coeff': 4, 'w': 20},
  {'widths': (2, 5, 10, 20), 'coeff': 5, 'w': 2},
  {'widths': (2, 5, 10, 20), 'coeff': 5, 'w': 5},
  {'widths': (2, 5, 10, 20), 'coeff': 5, 'w': 10},
  {'widths': (2, 5, 10, 20), 'coeff': 5, 'w': 20},
  {'widths': (2, 5, 10, 20), 'coeff': 6, 'w': 2},
  {'widths': (2, 5, 10, 20), 'coeff': 6, 'w': 5},
  {'widths': (2, 5, 10, 20), 'coeff': 6, 'w': 10},
  {'widths': (2, 5, 10, 20), 'coeff': 6, 'w': 20},
  {'widths': (2, 5, 10, 20), 'coeff': 7, 'w': 2},
  {'widths': (2, 5, 10, 20), 'coeff': 7, 'w': 5},
  {'widths': (2, 5, 10, 20), 'coeff': 7, 'w': 10},
  {'widths': (2, 5, 10, 20), 'coeff': 7, 'w': 20},
  {'widths': (2, 5, 10, 20), 'coeff': 8, 'w': 2},
  {'widths': (2, 5, 10, 20), 'coeff': 8, 'w': 5},
  {'widths': (2, 5, 10, 20), 'coeff': 8, 'w': 10},
  {'widths': (2, 5, 10, 20), 'coeff': 8, 'w': 20},
  {'widths': (2, 5, 10, 20), 'coeff': 9, 'w': 2},
  {'widths': (2, 5, 10, 20), 'coeff': 9, 'w': 5},
  {'widths': (2, 5, 10, 20), 'coeff': 9, 'w': 10},
  {'widths': (2, 5, 10, 20), 'coeff': 9, 'w': 20},
  {'widths': (2, 5, 10, 20), 'coeff': 10, 'w': 2},
  {'widths': (2, 5, 10, 20), 'coeff': 10, 'w': 5},
  {'widths': (2, 5, 10, 20), 'coeff': 10, 'w': 10},
  {'widths': (2, 5, 10, 20), 'coeff': 10, 'w': 20},
  {'widths': (2, 5, 10, 20), 'coeff': 11, 'w': 2},
  {'widths': (2, 5, 10, 20), 'coeff': 11, 'w': 5},
  {'widths': (2, 5, 10, 20), 'coeff': 11, 'w': 10},
  {'widths': (2, 5, 10, 20), 'coeff': 11, 'w': 20},
  {'widths': (2, 5, 10, 20), 'coeff': 12, 'w': 2},
  {'widths': (2, 5, 10, 20), 'coeff': 12, 'w': 5},
  {'widths': (2, 5, 10, 20), 'coeff': 12, 'w': 10},
  {'widths': (2, 5, 10, 20), 'coeff': 12, 'w': 20},
  {'widths': (2, 5, 10, 20), 'coeff': 13, 'w': 2},
  {'widths': (2, 5, 10, 20), 'coeff': 13, 'w': 5},
  {'widths': (2, 5, 10, 20), 'coeff': 13, 'w': 10},
  {'widths': (2, 5, 10, 20), 'coeff': 13, 'w': 20},
  {'widths': (2, 5, 10, 20), 'coeff': 14, 'w': 2},
  {'widths': (2, 5, 10, 20), 'coeff': 14, 'w': 5},
  {'widths': (2, 5, 10, 20), 'coeff': 14, 'w': 10},
  {'widths': (2, 5, 10, 20), 'coeff': 14, 'w': 20}],
 'spkt_welch_density': [{'coeff': 2}, {'coeff': 5}, {'coeff': 8}],
 'ar_coefficient': [{'coeff': 0, 'k': 10},
  {'coeff': 1, 'k': 10},
  {'coeff': 2, 'k': 10},
  {'coeff': 3, 'k': 10},
  {'coeff': 4, 'k': 10}],
 'change_quantiles': [{'ql': 0.0, 'qh': 0.2, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.2, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 0.2, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.2, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 0.4, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.4, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 0.4, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.4, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 0.6, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.6, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 0.6, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.6, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 0.8, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 0.8, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.8, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 1.0, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 1.0, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 1.0, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.2, 'qh': 0.2, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.2, 'qh': 0.2, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.2, 'qh': 0.2, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.2, 'qh': 0.2, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.2, 'qh': 0.4, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.2, 'qh': 0.4, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.2, 'qh': 0.4, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.2, 'qh': 0.4, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.2, 'qh': 0.6, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.2, 'qh': 0.6, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.2, 'qh': 0.6, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.2, 'qh': 0.6, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.2, 'qh': 0.8, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.2, 'qh': 0.8, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.2, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.2, 'qh': 1.0, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.2, 'qh': 1.0, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.2, 'qh': 1.0, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.4, 'qh': 0.2, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.4, 'qh': 0.2, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.4, 'qh': 0.2, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.4, 'qh': 0.2, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.4, 'qh': 0.4, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.4, 'qh': 0.4, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.4, 'qh': 0.4, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.4, 'qh': 0.4, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.4, 'qh': 0.6, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.4, 'qh': 0.6, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.4, 'qh': 0.6, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.4, 'qh': 0.6, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.4, 'qh': 0.8, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.4, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.4, 'qh': 0.8, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.4, 'qh': 0.8, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.4, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.4, 'qh': 1.0, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.4, 'qh': 1.0, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.4, 'qh': 1.0, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.6, 'qh': 0.2, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.6, 'qh': 0.2, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.6, 'qh': 0.2, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.6, 'qh': 0.2, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.6, 'qh': 0.4, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.6, 'qh': 0.4, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.6, 'qh': 0.4, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.6, 'qh': 0.4, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.6, 'qh': 0.6, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.6, 'qh': 0.6, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.6, 'qh': 0.6, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.6, 'qh': 0.6, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.6, 'qh': 0.8, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.6, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.6, 'qh': 0.8, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.6, 'qh': 0.8, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.6, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.6, 'qh': 1.0, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.6, 'qh': 1.0, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.6, 'qh': 1.0, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.8, 'qh': 0.2, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.8, 'qh': 0.2, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.8, 'qh': 0.2, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.8, 'qh': 0.2, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.8, 'qh': 0.4, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.8, 'qh': 0.4, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.8, 'qh': 0.4, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.8, 'qh': 0.4, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.8, 'qh': 0.6, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.8, 'qh': 0.6, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.8, 'qh': 0.6, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.8, 'qh': 0.6, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.8, 'qh': 0.8, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.8, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.8, 'qh': 0.8, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.8, 'qh': 0.8, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.8, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.8, 'qh': 1.0, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.8, 'qh': 1.0, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.8, 'qh': 1.0, 'isabs': True, 'f_agg': 'var'}],
 'fft_coefficient': [{'coeff': 0, 'attr': 'real'},
  {'coeff': 1, 'attr': 'real'},
  {'coeff': 2, 'attr': 'real'},
  {'coeff': 3, 'attr': 'real'},
  {'coeff': 4, 'attr': 'real'},
  {'coeff': 5, 'attr': 'real'},
  {'coeff': 6, 'attr': 'real'},
  {'coeff': 7, 'attr': 'real'},
  {'coeff': 8, 'attr': 'real'},
  {'coeff': 9, 'attr': 'real'},
  {'coeff': 10, 'attr': 'real'},
  {'coeff': 11, 'attr': 'real'},
  {'coeff': 12, 'attr': 'real'},
  {'coeff': 13, 'attr': 'real'},
  {'coeff': 14, 'attr': 'real'},
  {'coeff': 15, 'attr': 'real'},
  {'coeff': 16, 'attr': 'real'},
  {'coeff': 17, 'attr': 'real'},
  {'coeff': 18, 'attr': 'real'},
  {'coeff': 19, 'attr': 'real'},
  {'coeff': 20, 'attr': 'real'},
  {'coeff': 21, 'attr': 'real'},
  {'coeff': 22, 'attr': 'real'},
  {'coeff': 23, 'attr': 'real'},
  {'coeff': 24, 'attr': 'real'},
  {'coeff': 25, 'attr': 'real'},
  {'coeff': 26, 'attr': 'real'},
  {'coeff': 27, 'attr': 'real'},
  {'coeff': 28, 'attr': 'real'},
  {'coeff': 29, 'attr': 'real'},
  {'coeff': 30, 'attr': 'real'},
  {'coeff': 31, 'attr': 'real'},
  {'coeff': 32, 'attr': 'real'},
  {'coeff': 33, 'attr': 'real'},
  {'coeff': 34, 'attr': 'real'},
  {'coeff': 35, 'attr': 'real'},
  {'coeff': 36, 'attr': 'real'},
  {'coeff': 37, 'attr': 'real'},
  {'coeff': 38, 'attr': 'real'},
  {'coeff': 39, 'attr': 'real'},
  {'coeff': 40, 'attr': 'real'},
  {'coeff': 41, 'attr': 'real'},
  {'coeff': 42, 'attr': 'real'},
  {'coeff': 43, 'attr': 'real'},
  {'coeff': 44, 'attr': 'real'},
  {'coeff': 45, 'attr': 'real'},
  {'coeff': 46, 'attr': 'real'},
  {'coeff': 47, 'attr': 'real'},
  {'coeff': 48, 'attr': 'real'},
  {'coeff': 49, 'attr': 'real'},
  {'coeff': 50, 'attr': 'real'},
  {'coeff': 51, 'attr': 'real'},
  {'coeff': 52, 'attr': 'real'},
  {'coeff': 53, 'attr': 'real'},
  {'coeff': 54, 'attr': 'real'},
  {'coeff': 55, 'attr': 'real'},
  {'coeff': 56, 'attr': 'real'},
  {'coeff': 57, 'attr': 'real'},
  {'coeff': 58, 'attr': 'real'},
  {'coeff': 59, 'attr': 'real'},
  {'coeff': 60, 'attr': 'real'},
  {'coeff': 61, 'attr': 'real'},
  {'coeff': 62, 'attr': 'real'},
  {'coeff': 63, 'attr': 'real'},
  {'coeff': 64, 'attr': 'real'},
  {'coeff': 65, 'attr': 'real'},
  {'coeff': 66, 'attr': 'real'},
  {'coeff': 67, 'attr': 'real'},
  {'coeff': 68, 'attr': 'real'},
  {'coeff': 69, 'attr': 'real'},
  {'coeff': 70, 'attr': 'real'},
  {'coeff': 71, 'attr': 'real'},
  {'coeff': 72, 'attr': 'real'},
  {'coeff': 73, 'attr': 'real'},
  {'coeff': 74, 'attr': 'real'},
  {'coeff': 75, 'attr': 'real'},
  {'coeff': 76, 'attr': 'real'},
  {'coeff': 77, 'attr': 'real'},
  {'coeff': 78, 'attr': 'real'},
  {'coeff': 79, 'attr': 'real'},
  {'coeff': 80, 'attr': 'real'},
  {'coeff': 81, 'attr': 'real'},
  {'coeff': 82, 'attr': 'real'},
  {'coeff': 83, 'attr': 'real'},
  {'coeff': 84, 'attr': 'real'},
  {'coeff': 85, 'attr': 'real'},
  {'coeff': 86, 'attr': 'real'},
  {'coeff': 87, 'attr': 'real'},
  {'coeff': 88, 'attr': 'real'},
  {'coeff': 89, 'attr': 'real'},
  {'coeff': 90, 'attr': 'real'},
  {'coeff': 91, 'attr': 'real'},
  {'coeff': 92, 'attr': 'real'},
  {'coeff': 93, 'attr': 'real'},
  {'coeff': 94, 'attr': 'real'},
  {'coeff': 95, 'attr': 'real'},
  {'coeff': 96, 'attr': 'real'},
  {'coeff': 97, 'attr': 'real'},
  {'coeff': 98, 'attr': 'real'},
  {'coeff': 99, 'attr': 'real'},
  {'coeff': 0, 'attr': 'imag'},
  {'coeff': 1, 'attr': 'imag'},
  {'coeff': 2, 'attr': 'imag'},
  {'coeff': 3, 'attr': 'imag'},
  {'coeff': 4, 'attr': 'imag'},
  {'coeff': 5, 'attr': 'imag'},
  {'coeff': 6, 'attr': 'imag'},
  {'coeff': 7, 'attr': 'imag'},
  {'coeff': 8, 'attr': 'imag'},
  {'coeff': 9, 'attr': 'imag'},
  {'coeff': 10, 'attr': 'imag'},
  {'coeff': 11, 'attr': 'imag'},
  {'coeff': 12, 'attr': 'imag'},
  {'coeff': 13, 'attr': 'imag'},
  {'coeff': 14, 'attr': 'imag'},
  {'coeff': 15, 'attr': 'imag'},
  {'coeff': 16, 'attr': 'imag'},
  {'coeff': 17, 'attr': 'imag'},
  {'coeff': 18, 'attr': 'imag'},
  {'coeff': 19, 'attr': 'imag'},
  {'coeff': 20, 'attr': 'imag'},
  {'coeff': 21, 'attr': 'imag'},
  {'coeff': 22, 'attr': 'imag'},
  {'coeff': 23, 'attr': 'imag'},
  {'coeff': 24, 'attr': 'imag'},
  {'coeff': 25, 'attr': 'imag'},
  {'coeff': 26, 'attr': 'imag'},
  {'coeff': 27, 'attr': 'imag'},
  {'coeff': 28, 'attr': 'imag'},
  {'coeff': 29, 'attr': 'imag'},
  {'coeff': 30, 'attr': 'imag'},
  {'coeff': 31, 'attr': 'imag'},
  {'coeff': 32, 'attr': 'imag'},
  {'coeff': 33, 'attr': 'imag'},
  {'coeff': 34, 'attr': 'imag'},
  {'coeff': 35, 'attr': 'imag'},
  {'coeff': 36, 'attr': 'imag'},
  {'coeff': 37, 'attr': 'imag'},
  {'coeff': 38, 'attr': 'imag'},
  {'coeff': 39, 'attr': 'imag'},
  {'coeff': 40, 'attr': 'imag'},
  {'coeff': 41, 'attr': 'imag'},
  {'coeff': 42, 'attr': 'imag'},
  {'coeff': 43, 'attr': 'imag'},
  {'coeff': 44, 'attr': 'imag'},
  {'coeff': 45, 'attr': 'imag'},
  {'coeff': 46, 'attr': 'imag'},
  {'coeff': 47, 'attr': 'imag'},
  {'coeff': 48, 'attr': 'imag'},
  {'coeff': 49, 'attr': 'imag'},
  {'coeff': 50, 'attr': 'imag'},
  {'coeff': 51, 'attr': 'imag'},
  {'coeff': 52, 'attr': 'imag'},
  {'coeff': 53, 'attr': 'imag'},
  {'coeff': 54, 'attr': 'imag'},
  {'coeff': 55, 'attr': 'imag'},
  {'coeff': 56, 'attr': 'imag'},
  {'coeff': 57, 'attr': 'imag'},
  {'coeff': 58, 'attr': 'imag'},
  {'coeff': 59, 'attr': 'imag'},
  {'coeff': 60, 'attr': 'imag'},
  {'coeff': 61, 'attr': 'imag'},
  {'coeff': 62, 'attr': 'imag'},
  {'coeff': 63, 'attr': 'imag'},
  {'coeff': 64, 'attr': 'imag'},
  {'coeff': 65, 'attr': 'imag'},
  {'coeff': 66, 'attr': 'imag'},
  {'coeff': 67, 'attr': 'imag'},
  {'coeff': 68, 'attr': 'imag'},
  {'coeff': 69, 'attr': 'imag'},
  {'coeff': 70, 'attr': 'imag'},
  {'coeff': 71, 'attr': 'imag'},
  {'coeff': 72, 'attr': 'imag'},
  {'coeff': 73, 'attr': 'imag'},
  {'coeff': 74, 'attr': 'imag'},
  {'coeff': 75, 'attr': 'imag'},
  {'coeff': 76, 'attr': 'imag'},
  {'coeff': 77, 'attr': 'imag'},
  {'coeff': 78, 'attr': 'imag'},
  {'coeff': 79, 'attr': 'imag'},
  {'coeff': 80, 'attr': 'imag'},
  {'coeff': 81, 'attr': 'imag'},
  {'coeff': 82, 'attr': 'imag'},
  {'coeff': 83, 'attr': 'imag'},
  {'coeff': 84, 'attr': 'imag'},
  {'coeff': 85, 'attr': 'imag'},
  {'coeff': 86, 'attr': 'imag'},
  {'coeff': 87, 'attr': 'imag'},
  {'coeff': 88, 'attr': 'imag'},
  {'coeff': 89, 'attr': 'imag'},
  {'coeff': 90, 'attr': 'imag'},
  {'coeff': 91, 'attr': 'imag'},
  {'coeff': 92, 'attr': 'imag'},
  {'coeff': 93, 'attr': 'imag'},
  {'coeff': 94, 'attr': 'imag'},
  {'coeff': 95, 'attr': 'imag'},
  {'coeff': 96, 'attr': 'imag'},
  {'coeff': 97, 'attr': 'imag'},
  {'coeff': 98, 'attr': 'imag'},
  {'coeff': 99, 'attr': 'imag'},
  {'coeff': 0, 'attr': 'abs'},
  {'coeff': 1, 'attr': 'abs'},
  {'coeff': 2, 'attr': 'abs'},
  {'coeff': 3, 'attr': 'abs'},
  {'coeff': 4, 'attr': 'abs'},
  {'coeff': 5, 'attr': 'abs'},
  {'coeff': 6, 'attr': 'abs'},
  {'coeff': 7, 'attr': 'abs'},
  {'coeff': 8, 'attr': 'abs'},
  {'coeff': 9, 'attr': 'abs'},
  {'coeff': 10, 'attr': 'abs'},
  {'coeff': 11, 'attr': 'abs'},
  {'coeff': 12, 'attr': 'abs'},
  {'coeff': 13, 'attr': 'abs'},
  {'coeff': 14, 'attr': 'abs'},
  {'coeff': 15, 'attr': 'abs'},
  {'coeff': 16, 'attr': 'abs'},
  {'coeff': 17, 'attr': 'abs'},
  {'coeff': 18, 'attr': 'abs'},
  {'coeff': 19, 'attr': 'abs'},
  {'coeff': 20, 'attr': 'abs'},
  {'coeff': 21, 'attr': 'abs'},
  {'coeff': 22, 'attr': 'abs'},
  {'coeff': 23, 'attr': 'abs'},
  {'coeff': 24, 'attr': 'abs'},
  {'coeff': 25, 'attr': 'abs'},
  {'coeff': 26, 'attr': 'abs'},
  {'coeff': 27, 'attr': 'abs'},
  {'coeff': 28, 'attr': 'abs'},
  {'coeff': 29, 'attr': 'abs'},
  {'coeff': 30, 'attr': 'abs'},
  {'coeff': 31, 'attr': 'abs'},
  {'coeff': 32, 'attr': 'abs'},
  {'coeff': 33, 'attr': 'abs'},
  {'coeff': 34, 'attr': 'abs'},
  {'coeff': 35, 'attr': 'abs'},
  {'coeff': 36, 'attr': 'abs'},
  {'coeff': 37, 'attr': 'abs'},
  {'coeff': 38, 'attr': 'abs'},
  {'coeff': 39, 'attr': 'abs'},
  {'coeff': 40, 'attr': 'abs'},
  {'coeff': 41, 'attr': 'abs'},
  {'coeff': 42, 'attr': 'abs'},
  {'coeff': 43, 'attr': 'abs'},
  {'coeff': 44, 'attr': 'abs'},
  {'coeff': 45, 'attr': 'abs'},
  {'coeff': 46, 'attr': 'abs'},
  {'coeff': 47, 'attr': 'abs'},
  {'coeff': 48, 'attr': 'abs'},
  {'coeff': 49, 'attr': 'abs'},
  {'coeff': 50, 'attr': 'abs'},
  {'coeff': 51, 'attr': 'abs'},
  {'coeff': 52, 'attr': 'abs'},
  {'coeff': 53, 'attr': 'abs'},
  {'coeff': 54, 'attr': 'abs'},
  {'coeff': 55, 'attr': 'abs'},
  {'coeff': 56, 'attr': 'abs'},
  {'coeff': 57, 'attr': 'abs'},
  {'coeff': 58, 'attr': 'abs'},
  {'coeff': 59, 'attr': 'abs'},
  {'coeff': 60, 'attr': 'abs'},
  {'coeff': 61, 'attr': 'abs'},
  {'coeff': 62, 'attr': 'abs'},
  {'coeff': 63, 'attr': 'abs'},
  {'coeff': 64, 'attr': 'abs'},
  {'coeff': 65, 'attr': 'abs'},
  {'coeff': 66, 'attr': 'abs'},
  {'coeff': 67, 'attr': 'abs'},
  {'coeff': 68, 'attr': 'abs'},
  {'coeff': 69, 'attr': 'abs'},
  {'coeff': 70, 'attr': 'abs'},
  {'coeff': 71, 'attr': 'abs'},
  {'coeff': 72, 'attr': 'abs'},
  {'coeff': 73, 'attr': 'abs'},
  {'coeff': 74, 'attr': 'abs'},
  {'coeff': 75, 'attr': 'abs'},
  {'coeff': 76, 'attr': 'abs'},
  {'coeff': 77, 'attr': 'abs'},
  {'coeff': 78, 'attr': 'abs'},
  {'coeff': 79, 'attr': 'abs'},
  {'coeff': 80, 'attr': 'abs'},
  {'coeff': 81, 'attr': 'abs'},
  {'coeff': 82, 'attr': 'abs'},
  {'coeff': 83, 'attr': 'abs'},
  {'coeff': 84, 'attr': 'abs'},
  {'coeff': 85, 'attr': 'abs'},
  {'coeff': 86, 'attr': 'abs'},
  {'coeff': 87, 'attr': 'abs'},
  {'coeff': 88, 'attr': 'abs'},
  {'coeff': 89, 'attr': 'abs'},
  {'coeff': 90, 'attr': 'abs'},
  {'coeff': 91, 'attr': 'abs'},
  {'coeff': 92, 'attr': 'abs'},
  {'coeff': 93, 'attr': 'abs'},
  {'coeff': 94, 'attr': 'abs'},
  {'coeff': 95, 'attr': 'abs'},
  {'coeff': 96, 'attr': 'abs'},
  {'coeff': 97, 'attr': 'abs'},
  {'coeff': 98, 'attr': 'abs'},
  {'coeff': 99, 'attr': 'abs'},
  {'coeff': 0, 'attr': 'angle'},
  {'coeff': 1, 'attr': 'angle'},
  {'coeff': 2, 'attr': 'angle'},
  {'coeff': 3, 'attr': 'angle'},
  {'coeff': 4, 'attr': 'angle'},
  {'coeff': 5, 'attr': 'angle'},
  {'coeff': 6, 'attr': 'angle'},
  {'coeff': 7, 'attr': 'angle'},
  {'coeff': 8, 'attr': 'angle'},
  {'coeff': 9, 'attr': 'angle'},
  {'coeff': 10, 'attr': 'angle'},
  {'coeff': 11, 'attr': 'angle'},
  {'coeff': 12, 'attr': 'angle'},
  {'coeff': 13, 'attr': 'angle'},
  {'coeff': 14, 'attr': 'angle'},
  {'coeff': 15, 'attr': 'angle'},
  {'coeff': 16, 'attr': 'angle'},
  {'coeff': 17, 'attr': 'angle'},
  {'coeff': 18, 'attr': 'angle'},
  {'coeff': 19, 'attr': 'angle'},
  {'coeff': 20, 'attr': 'angle'},
  {'coeff': 21, 'attr': 'angle'},
  {'coeff': 22, 'attr': 'angle'},
  {'coeff': 23, 'attr': 'angle'},
  {'coeff': 24, 'attr': 'angle'},
  {'coeff': 25, 'attr': 'angle'},
  {'coeff': 26, 'attr': 'angle'},
  {'coeff': 27, 'attr': 'angle'},
  {'coeff': 28, 'attr': 'angle'},
  {'coeff': 29, 'attr': 'angle'},
  {'coeff': 30, 'attr': 'angle'},
  {'coeff': 31, 'attr': 'angle'},
  {'coeff': 32, 'attr': 'angle'},
  {'coeff': 33, 'attr': 'angle'},
  {'coeff': 34, 'attr': 'angle'},
  {'coeff': 35, 'attr': 'angle'},
  {'coeff': 36, 'attr': 'angle'},
  {'coeff': 37, 'attr': 'angle'},
  {'coeff': 38, 'attr': 'angle'},
  {'coeff': 39, 'attr': 'angle'},
  {'coeff': 40, 'attr': 'angle'},
  {'coeff': 41, 'attr': 'angle'},
  {'coeff': 42, 'attr': 'angle'},
  {'coeff': 43, 'attr': 'angle'},
  {'coeff': 44, 'attr': 'angle'},
  {'coeff': 45, 'attr': 'angle'},
  {'coeff': 46, 'attr': 'angle'},
  {'coeff': 47, 'attr': 'angle'},
  {'coeff': 48, 'attr': 'angle'},
  {'coeff': 49, 'attr': 'angle'},
  {'coeff': 50, 'attr': 'angle'},
  {'coeff': 51, 'attr': 'angle'},
  {'coeff': 52, 'attr': 'angle'},
  {'coeff': 53, 'attr': 'angle'},
  {'coeff': 54, 'attr': 'angle'},
  {'coeff': 55, 'attr': 'angle'},
  {'coeff': 56, 'attr': 'angle'},
  {'coeff': 57, 'attr': 'angle'},
  {'coeff': 58, 'attr': 'angle'},
  {'coeff': 59, 'attr': 'angle'},
  {'coeff': 60, 'attr': 'angle'},
  {'coeff': 61, 'attr': 'angle'},
  {'coeff': 62, 'attr': 'angle'},
  {'coeff': 63, 'attr': 'angle'},
  {'coeff': 64, 'attr': 'angle'},
  {'coeff': 65, 'attr': 'angle'},
  {'coeff': 66, 'attr': 'angle'},
  {'coeff': 67, 'attr': 'angle'},
  {'coeff': 68, 'attr': 'angle'},
  {'coeff': 69, 'attr': 'angle'},
  {'coeff': 70, 'attr': 'angle'},
  {'coeff': 71, 'attr': 'angle'},
  {'coeff': 72, 'attr': 'angle'},
  {'coeff': 73, 'attr': 'angle'},
  {'coeff': 74, 'attr': 'angle'},
  {'coeff': 75, 'attr': 'angle'},
  {'coeff': 76, 'attr': 'angle'},
  {'coeff': 77, 'attr': 'angle'},
  {'coeff': 78, 'attr': 'angle'},
  {'coeff': 79, 'attr': 'angle'},
  {'coeff': 80, 'attr': 'angle'},
  {'coeff': 81, 'attr': 'angle'},
  {'coeff': 82, 'attr': 'angle'},
  {'coeff': 83, 'attr': 'angle'},
  {'coeff': 84, 'attr': 'angle'},
  {'coeff': 85, 'attr': 'angle'},
  {'coeff': 86, 'attr': 'angle'},
  {'coeff': 87, 'attr': 'angle'},
  {'coeff': 88, 'attr': 'angle'},
  {'coeff': 89, 'attr': 'angle'},
  {'coeff': 90, 'attr': 'angle'},
  {'coeff': 91, 'attr': 'angle'},
  {'coeff': 92, 'attr': 'angle'},
  {'coeff': 93, 'attr': 'angle'},
  {'coeff': 94, 'attr': 'angle'},
  {'coeff': 95, 'attr': 'angle'},
  {'coeff': 96, 'attr': 'angle'},
  {'coeff': 97, 'attr': 'angle'},
  {'coeff': 98, 'attr': 'angle'},
  {'coeff': 99, 'attr': 'angle'}],
 'fft_aggregated': [{'aggtype': 'centroid'},
  {'aggtype': 'variance'},
  {'aggtype': 'skew'},
  {'aggtype': 'kurtosis'}],
 'value_count': [{'value': 0}, {'value': 1}, {'value': -1}],
 'range_count': [{'min': -1, 'max': 1},
  {'min': 1000000000000.0, 'max': 0},
  {'min': 0, 'max': 1000000000000.0}],
 'approximate_entropy': [{'m': 2, 'r': 0.1},
  {'m': 2, 'r': 0.3},
  {'m': 2, 'r': 0.5},
  {'m': 2, 'r': 0.7},
  {'m': 2, 'r': 0.9}],
 'friedrich_coefficients': [{'coeff': 0, 'm': 3, 'r': 30},
  {'coeff': 1, 'm': 3, 'r': 30},
  {'coeff': 2, 'm': 3, 'r': 30},
  {'coeff': 3, 'm': 3, 'r': 30}],
 'max_langevin_fixed_point': [{'m': 3, 'r': 30}],
 'linear_trend': [{'attr': 'pvalue'},
  {'attr': 'rvalue'},
  {'attr': 'intercept'},
  {'attr': 'slope'},
  {'attr': 'stderr'}],
 'agg_linear_trend': [{'attr': 'rvalue', 'chunk_len': 5, 'f_agg': 'max'},
  {'attr': 'rvalue', 'chunk_len': 5, 'f_agg': 'min'},
  {'attr': 'rvalue', 'chunk_len': 5, 'f_agg': 'mean'},
  {'attr': 'rvalue', 'chunk_len': 5, 'f_agg': 'var'},
  {'attr': 'rvalue', 'chunk_len': 10, 'f_agg': 'max'},
  {'attr': 'rvalue', 'chunk_len': 10, 'f_agg': 'min'},
  {'attr': 'rvalue', 'chunk_len': 10, 'f_agg': 'mean'},
  {'attr': 'rvalue', 'chunk_len': 10, 'f_agg': 'var'},
  {'attr': 'rvalue', 'chunk_len': 50, 'f_agg': 'max'},
  {'attr': 'rvalue', 'chunk_len': 50, 'f_agg': 'min'},
  {'attr': 'rvalue', 'chunk_len': 50, 'f_agg': 'mean'},
  {'attr': 'rvalue', 'chunk_len': 50, 'f_agg': 'var'},
  {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'max'},
  {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'min'},
  {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'mean'},
  {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'var'},
  {'attr': 'intercept', 'chunk_len': 10, 'f_agg': 'max'},
  {'attr': 'intercept', 'chunk_len': 10, 'f_agg': 'min'},
  {'attr': 'intercept', 'chunk_len': 10, 'f_agg': 'mean'},
  {'attr': 'intercept', 'chunk_len': 10, 'f_agg': 'var'},
  {'attr': 'intercept', 'chunk_len': 50, 'f_agg': 'max'},
  {'attr': 'intercept', 'chunk_len': 50, 'f_agg': 'min'},
  {'attr': 'intercept', 'chunk_len': 50, 'f_agg': 'mean'},
  {'attr': 'intercept', 'chunk_len': 50, 'f_agg': 'var'},
  {'attr': 'slope', 'chunk_len': 5, 'f_agg': 'max'},
  {'attr': 'slope', 'chunk_len': 5, 'f_agg': 'min'},
  {'attr': 'slope', 'chunk_len': 5, 'f_agg': 'mean'},
  {'attr': 'slope', 'chunk_len': 5, 'f_agg': 'var'},
  {'attr': 'slope', 'chunk_len': 10, 'f_agg': 'max'},
  {'attr': 'slope', 'chunk_len': 10, 'f_agg': 'min'},
  {'attr': 'slope', 'chunk_len': 10, 'f_agg': 'mean'},
  {'attr': 'slope', 'chunk_len': 10, 'f_agg': 'var'},
  {'attr': 'slope', 'chunk_len': 50, 'f_agg': 'max'},
  {'attr': 'slope', 'chunk_len': 50, 'f_agg': 'min'},
  {'attr': 'slope', 'chunk_len': 50, 'f_agg': 'mean'},
  {'attr': 'slope', 'chunk_len': 50, 'f_agg': 'var'},
  {'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'max'},
  {'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'min'},
  {'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'mean'},
  {'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'var'},
  {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'max'},
  {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'min'},
  {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'mean'},
  {'attr': 'stderr', 'chunk_len': 10, 'f_agg': 'var'},
  {'attr': 'stderr', 'chunk_len': 50, 'f_agg': 'max'},
  {'attr': 'stderr', 'chunk_len': 50, 'f_agg': 'min'},
  {'attr': 'stderr', 'chunk_len': 50, 'f_agg': 'mean'},
  {'attr': 'stderr', 'chunk_len': 50, 'f_agg': 'var'}],
 'augmented_dickey_fuller': [{'attr': 'teststat'},
  {'attr': 'pvalue'},
  {'attr': 'usedlag'}],
 'number_crossing_m': [{'m': 0}, {'m': -1}, {'m': 1}],
 'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
  {'num_segments': 10, 'segment_focus': 1},
  {'num_segments': 10, 'segment_focus': 2},
  {'num_segments': 10, 'segment_focus': 3},
  {'num_segments': 10, 'segment_focus': 4},
  {'num_segments': 10, 'segment_focus': 5},
  {'num_segments': 10, 'segment_focus': 6},
  {'num_segments': 10, 'segment_focus': 7},
  {'num_segments': 10, 'segment_focus': 8},
  {'num_segments': 10, 'segment_focus': 9}],
 'ratio_beyond_r_sigma': [{'r': 0.5},
  {'r': 1},
  {'r': 1.5},
  {'r': 2},
  {'r': 2.5},
  {'r': 3},
  {'r': 5},
  {'r': 6},
  {'r': 7},
  {'r': 10}]}

### 1rst Selection

In [0]:
settings={
'standard_deviation': None,
 'variance': None,
 'skewness': None,
 'kurtosis': None,
 'absolute_sum_of_changes': None,

'last_location_of_maximum': None,
 'first_location_of_maximum': None,
 'last_location_of_minimum': None,
'first_location_of_minimum': None,
'count_above_mean': None,
 'count_below_mean': None,
 'maximum': None,
 'minimum': None,

    
 'c3': [{'lag': 100}, {'lag': 2000}, {'lag': 3000},{'lag': 10000}],

'number_peaks': [{'n': 1}, {'n': 5},{'n': 100},{'n': 1000}],
'fft_coefficient': 
 [{'coeff': 0, 'attr': 'real'},
  {'coeff': 1, 'attr': 'real'},
  {'coeff': 2, 'attr': 'real'},
  {'coeff': 3, 'attr': 'real'},
  {'coeff': 4, 'attr': 'real'},

  {'coeff': 0, 'attr': 'imag'},
  {'coeff': 1, 'attr': 'imag'},
  {'coeff': 2, 'attr': 'imag'},
  {'coeff': 3, 'attr': 'imag'},
  {'coeff': 4, 'attr': 'imag'},
  {'coeff': 5, 'attr': 'imag'},

  {'coeff': 0, 'attr': 'abs'},
  {'coeff': 1, 'attr': 'abs'},
  {'coeff': 2, 'attr': 'abs'},
  {'coeff': 3, 'attr': 'abs'},
  {'coeff': 4, 'attr': 'abs'},
  {'coeff': 5, 'attr': 'abs'},

  {'coeff': 0, 'attr': 'angle'},
  {'coeff': 1, 'attr': 'angle'},
  {'coeff': 2, 'attr': 'angle'},
  {'coeff': 3, 'attr': 'angle'},
  {'coeff': 4, 'attr': 'angle'},
  {'coeff': 5, 'attr': 'angle'},
],
 'agg_linear_trend': [
     {'attr': 'rvalue', 'chunk_len': 500, 'f_agg': 'max'},
  {'attr': 'rvalue', 'chunk_len': 500, 'f_agg': 'min'},
  {'attr': 'rvalue', 'chunk_len': 500, 'f_agg': 'mean'},
  {'attr': 'rvalue', 'chunk_len': 500, 'f_agg': 'var'},
  {'attr': 'rvalue', 'chunk_len': 1000, 'f_agg': 'max'},
  {'attr': 'rvalue', 'chunk_len': 1000, 'f_agg': 'min'},
  {'attr': 'rvalue', 'chunk_len': 1000, 'f_agg': 'mean'},
  {'attr': 'rvalue', 'chunk_len': 1000, 'f_agg': 'var'},
  {'attr': 'rvalue', 'chunk_len': 5000, 'f_agg': 'max'},
  {'attr': 'rvalue', 'chunk_len': 5000, 'f_agg': 'min'},
  {'attr': 'rvalue', 'chunk_len': 5000, 'f_agg': 'mean'},
  {'attr': 'rvalue', 'chunk_len': 5000, 'f_agg': 'var'},
  {'attr': 'intercept', 'chunk_len': 500, 'f_agg': 'max'},
  {'attr': 'intercept', 'chunk_len': 500, 'f_agg': 'min'},
  {'attr': 'intercept', 'chunk_len': 500, 'f_agg': 'mean'},
  {'attr': 'intercept', 'chunk_len': 500, 'f_agg': 'var'},
  {'attr': 'intercept', 'chunk_len': 1000, 'f_agg': 'max'},
  {'attr': 'intercept', 'chunk_len': 1000, 'f_agg': 'min'},
  {'attr': 'intercept', 'chunk_len': 1000, 'f_agg': 'mean'},
  {'attr': 'intercept', 'chunk_len': 1000, 'f_agg': 'var'},
  {'attr': 'intercept', 'chunk_len': 5000, 'f_agg': 'max'},
  {'attr': 'intercept', 'chunk_len': 5000, 'f_agg': 'min'},
  {'attr': 'intercept', 'chunk_len': 5000, 'f_agg': 'mean'},
  {'attr': 'intercept', 'chunk_len': 5000, 'f_agg': 'var'},
  {'attr': 'slope', 'chunk_len': 500, 'f_agg': 'max'},
  {'attr': 'slope', 'chunk_len': 500, 'f_agg': 'min'},
  {'attr': 'slope', 'chunk_len': 500, 'f_agg': 'mean'},
  {'attr': 'slope', 'chunk_len': 500, 'f_agg': 'var'},
  {'attr': 'slope', 'chunk_len': 1000, 'f_agg': 'max'},
  {'attr': 'slope', 'chunk_len': 1000, 'f_agg': 'min'},
  {'attr': 'slope', 'chunk_len': 1000, 'f_agg': 'mean'},
  {'attr': 'slope', 'chunk_len': 1000, 'f_agg': 'var'},
  {'attr': 'slope', 'chunk_len': 5000, 'f_agg': 'max'},
  {'attr': 'slope', 'chunk_len': 5000, 'f_agg': 'min'},
  {'attr': 'slope', 'chunk_len': 5000, 'f_agg': 'mean'},
  {'attr': 'slope', 'chunk_len': 5000, 'f_agg': 'var'},
  {'attr': 'stderr', 'chunk_len': 500, 'f_agg': 'max'},
  {'attr': 'stderr', 'chunk_len': 500, 'f_agg': 'min'},
  {'attr': 'stderr', 'chunk_len': 500, 'f_agg': 'mean'},
  {'attr': 'stderr', 'chunk_len': 500, 'f_agg': 'var'},
  {'attr': 'stderr', 'chunk_len': 1000, 'f_agg': 'max'},
  {'attr': 'stderr', 'chunk_len': 1000, 'f_agg': 'min'},
  {'attr': 'stderr', 'chunk_len': 1000, 'f_agg': 'mean'},
  {'attr': 'stderr', 'chunk_len': 1000, 'f_agg': 'var'},
  {'attr': 'stderr', 'chunk_len': 5000, 'f_agg': 'max'},
  {'attr': 'stderr', 'chunk_len': 5000, 'f_agg': 'min'},
  {'attr': 'stderr', 'chunk_len': 5000, 'f_agg': 'mean'},
  {'attr': 'stderr', 'chunk_len': 5000, 'f_agg': 'var'}],
 'index_mass_quantile': [{'q': 0.1},
  {'q': 0.2},
  {'q': 0.3},
  {'q': 0.4},
  {'q': 0.6},
  {'q': 0.7},
  {'q': 0.8},
  {'q': 0.9}],

 'spkt_welch_density': [{'coeff': 2}, {'coeff': 5}, {'coeff': 8}],
 'ar_coefficient': [{'coeff': 0, 'k': 10},
  {'coeff': 1, 'k': 10},
  {'coeff': 2, 'k': 10},
  {'coeff': 3, 'k': 10},
  {'coeff': 4, 'k': 10}],

 'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
  {'num_segments': 10, 'segment_focus': 1},
  {'num_segments': 10, 'segment_focus': 2},
  {'num_segments': 10, 'segment_focus': 3},
  {'num_segments': 10, 'segment_focus': 4},
  {'num_segments': 10, 'segment_focus': 5},
  {'num_segments': 10, 'segment_focus': 6},
  {'num_segments': 10, 'segment_focus': 7},
  {'num_segments': 10, 'segment_focus': 8},
  {'num_segments': 10, 'segment_focus': 9}],
 'ratio_beyond_r_sigma': [{'r': 0.5},
  {'r': 1},
  {'r': 1.5},
  {'r': 2},
  {'r': 2.5},
  {'r': 3},
  {'r': 5},
  {'r': 6},
  {'r': 7},
  {'r': 10}],
'max_langevin_fixed_point': [{'m': 3, 'r': 30}],
    'change_quantiles': [{'ql': 0.0, 'qh': 0.2, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.2, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 0.2, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.2, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 0.4, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.4, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 0.4, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.4, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 0.6, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.6, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 0.6, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.6, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 0.8, 'isabs': False, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 0.8, 'isabs': True, 'f_agg': 'mean'},
  {'ql': 0.0, 'qh': 0.8, 'isabs': True, 'f_agg': 'var'},
  {'ql': 0.0, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'}]

}

### 2nd Selection

In [0]:
settings={ 
  'standard_deviation': None,    
  'variance': None,
  'skewness': None,
  'kurtosis': None,
  'count_above_mean': None,
  'count_below_mean': None,
  'last_location_of_maximum': None,
  'first_location_of_maximum': None,
  'last_location_of_minimum': None,
  'first_location_of_minimum': None,
  'maximum': None,
  'minimum': None,

  'c3': [{'lag': 100}, {'lag': 2000}, {'lag': 3000},{'lag': 10000}],

  'number_peaks': 
    [{'n': 1},
    {'n': 2},
    {'n': 3},
    {'n': 5},
    {'n': 8},
    {'n': 13},
    {'n': 21},
    {'n': 100},
    {'n': 5000}
    ],
    
'fft_coefficient': 
 [{'coeff': 0, 'attr': 'real'},
  {'coeff': 1, 'attr': 'real'},
  {'coeff': 2, 'attr': 'real'},
  {'coeff': 3, 'attr': 'real'},
  {'coeff': 4, 'attr': 'real'},

  {'coeff': 0, 'attr': 'imag'},
  {'coeff': 1, 'attr': 'imag'},
  {'coeff': 2, 'attr': 'imag'},
  {'coeff': 3, 'attr': 'imag'},
  {'coeff': 4, 'attr': 'imag'},
  {'coeff': 5, 'attr': 'imag'},

  {'coeff': 0, 'attr': 'abs'},
  {'coeff': 1, 'attr': 'abs'},
  {'coeff': 2, 'attr': 'abs'},
  {'coeff': 3, 'attr': 'abs'},
  {'coeff': 4, 'attr': 'abs'},
  {'coeff': 5, 'attr': 'abs'},

  {'coeff': 0, 'attr': 'angle'},
  {'coeff': 1, 'attr': 'angle'},
  {'coeff': 2, 'attr': 'angle'},
  {'coeff': 3, 'attr': 'angle'},
  {'coeff': 4, 'attr': 'angle'},
  {'coeff': 5, 'attr': 'angle'},],
    
  'agg_linear_trend': 
    [{'attr': 'rvalue', 'chunk_len': 500, 'f_agg': 'max'}, 
    {'attr': 'rvalue', 'chunk_len': 500, 'f_agg': 'min'},
    {'attr': 'rvalue', 'chunk_len': 500, 'f_agg': 'mean'},
    {'attr': 'rvalue', 'chunk_len': 500, 'f_agg': 'var'},
    {'attr': 'rvalue', 'chunk_len': 1000, 'f_agg': 'max'},
    {'attr': 'rvalue', 'chunk_len': 1000, 'f_agg': 'min'},
    {'attr': 'rvalue', 'chunk_len': 1000, 'f_agg': 'mean'},
    {'attr': 'rvalue', 'chunk_len': 1000, 'f_agg': 'var'},
    {'attr': 'rvalue', 'chunk_len': 5000, 'f_agg': 'max'},
    {'attr': 'rvalue', 'chunk_len': 5000, 'f_agg': 'min'},
    {'attr': 'rvalue', 'chunk_len': 5000, 'f_agg': 'mean'},
    {'attr': 'rvalue', 'chunk_len': 5000, 'f_agg': 'var'},
    {'attr': 'intercept', 'chunk_len': 500, 'f_agg': 'max'},
    {'attr': 'intercept', 'chunk_len': 500, 'f_agg': 'min'},
    {'attr': 'intercept', 'chunk_len': 500, 'f_agg': 'mean'},
    {'attr': 'intercept', 'chunk_len': 500, 'f_agg': 'var'},
    {'attr': 'intercept', 'chunk_len': 1000, 'f_agg': 'max'},
    {'attr': 'intercept', 'chunk_len': 1000, 'f_agg': 'min'},
    {'attr': 'intercept', 'chunk_len': 1000, 'f_agg': 'mean'},
    {'attr': 'intercept', 'chunk_len': 1000, 'f_agg': 'var'},
    {'attr': 'intercept', 'chunk_len': 5000, 'f_agg': 'max'},
    {'attr': 'intercept', 'chunk_len': 5000, 'f_agg': 'min'},
    {'attr': 'intercept', 'chunk_len': 5000, 'f_agg': 'mean'},
    {'attr': 'intercept', 'chunk_len': 5000, 'f_agg': 'var'},
    {'attr': 'slope', 'chunk_len': 500, 'f_agg': 'max'},
    {'attr': 'slope', 'chunk_len': 500, 'f_agg': 'min'},
    {'attr': 'slope', 'chunk_len': 500, 'f_agg': 'mean'},
    {'attr': 'slope', 'chunk_len': 500, 'f_agg': 'var'},
    {'attr': 'slope', 'chunk_len': 1000, 'f_agg': 'max'},
    {'attr': 'slope', 'chunk_len': 1000, 'f_agg': 'min'},
    {'attr': 'slope', 'chunk_len': 1000, 'f_agg': 'mean'},
    {'attr': 'slope', 'chunk_len': 1000, 'f_agg': 'var'},
    {'attr': 'slope', 'chunk_len': 5000, 'f_agg': 'max'},
    {'attr': 'slope', 'chunk_len': 5000, 'f_agg': 'min'},
    {'attr': 'slope', 'chunk_len': 5000, 'f_agg': 'mean'},
    {'attr': 'slope', 'chunk_len': 5000, 'f_agg': 'var'},
    {'attr': 'stderr', 'chunk_len': 500, 'f_agg': 'max'},
    {'attr': 'stderr', 'chunk_len': 500, 'f_agg': 'min'},
    {'attr': 'stderr', 'chunk_len': 500, 'f_agg': 'mean'},
    {'attr': 'stderr', 'chunk_len': 500, 'f_agg': 'var'},
    {'attr': 'stderr', 'chunk_len': 1000, 'f_agg': 'max'},
    {'attr': 'stderr', 'chunk_len': 1000, 'f_agg': 'min'},
    {'attr': 'stderr', 'chunk_len': 1000, 'f_agg': 'mean'},
    {'attr': 'stderr', 'chunk_len': 1000, 'f_agg': 'var'},
    {'attr': 'stderr', 'chunk_len': 5000, 'f_agg': 'max'},
    {'attr': 'stderr', 'chunk_len': 5000, 'f_agg': 'min'},
    {'attr': 'stderr', 'chunk_len': 5000, 'f_agg': 'mean'},
    {'attr': 'stderr', 'chunk_len': 5000, 'f_agg': 'var'}],
    
  'index_mass_quantile':
    [{'q': 0.1},     
    {'q': 0.2},
    {'q': 0.3},
    {'q': 0.4},
    {'q': 0.6},
    {'q': 0.7},
    {'q': 0.8},
    {'q': 0.9}],

  'spkt_welch_density': [{'coeff': 2}, {'coeff': 5}, {'coeff': 8}],

  'ar_coefficient': 
    [{'coeff': 0, 'k': 10},
    {'coeff': 1, 'k': 10},
    {'coeff': 2, 'k': 10},
    {'coeff': 3, 'k': 10},
    {'coeff': 4, 'k': 10}],

  'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
    {'num_segments': 10, 'segment_focus': 1},
    {'num_segments': 10, 'segment_focus': 2},
    {'num_segments': 10, 'segment_focus': 3},
    {'num_segments': 10, 'segment_focus': 4},
    {'num_segments': 10, 'segment_focus': 5},
    {'num_segments': 10, 'segment_focus': 6},
    {'num_segments': 10, 'segment_focus': 7},
    {'num_segments': 10, 'segment_focus': 8},
    {'num_segments': 10, 'segment_focus': 9}],
    
  'ratio_beyond_r_sigma': 
    [{'r': 0.5},
    {'r': 1},
    {'r': 1.5},
    {'r': 2},
    {'r': 2.5},
    {'r': 3},
    {'r': 5},
    {'r': 6},
    {'r': 7},
    {'r': 10}],

  'max_langevin_fixed_point': [{'m': 3, 'r': 30}],

  }

peak={
  'number_peaks': 
    [{'n': 1},
    {'n': 2},
    {'n': 3},
    {'n': 5},
    {'n': 8},
    {'n': 13},
    {'n': 21}]
}

In [0]:
settings={
'standard_deviation': None,
 'variance': None,
 'skewness': None,
 'kurtosis': None,

 'count_above_mean': None,
 'count_below_mean': None,
'last_location_of_maximum': None,
 'first_location_of_maximum': None,
 'last_location_of_minimum': None,
'first_location_of_minimum': None,

 'maximum': None,
 'minimum': None,

 'c3': [{'lag': 100}, {'lag': 2000}, {'lag': 3000},{'lag': 10000}],

'number_peaks': [{'n': 1}, 
                  {'n': 3}, 
                  {'n': 5},{'n': 100},{'n': 5000}],
'fft_coefficient': 
 [{'coeff': 0, 'attr': 'real'},
  {'coeff': 1, 'attr': 'real'},
  {'coeff': 2, 'attr': 'real'},
  {'coeff': 3, 'attr': 'real'},
  {'coeff': 4, 'attr': 'real'},

  {'coeff': 0, 'attr': 'imag'},
  {'coeff': 1, 'attr': 'imag'},
  {'coeff': 2, 'attr': 'imag'},
  {'coeff': 3, 'attr': 'imag'},
  {'coeff': 4, 'attr': 'imag'},
  {'coeff': 5, 'attr': 'imag'},

  {'coeff': 0, 'attr': 'abs'},
  {'coeff': 1, 'attr': 'abs'},
  {'coeff': 2, 'attr': 'abs'},
  {'coeff': 3, 'attr': 'abs'},
  {'coeff': 4, 'attr': 'abs'},
  {'coeff': 5, 'attr': 'abs'},

  {'coeff': 0, 'attr': 'angle'},
  {'coeff': 1, 'attr': 'angle'},
  {'coeff': 2, 'attr': 'angle'},
  {'coeff': 3, 'attr': 'angle'},
  {'coeff': 4, 'attr': 'angle'},
  {'coeff': 5, 'attr': 'angle'},
],
 'agg_linear_trend': [
     {'attr': 'rvalue', 'chunk_len': 500, 'f_agg': 'max'},
  {'attr': 'rvalue', 'chunk_len': 500, 'f_agg': 'min'},
  {'attr': 'rvalue', 'chunk_len': 500, 'f_agg': 'mean'},
  {'attr': 'rvalue', 'chunk_len': 500, 'f_agg': 'var'},
  {'attr': 'rvalue', 'chunk_len': 1000, 'f_agg': 'max'},
  {'attr': 'rvalue', 'chunk_len': 1000, 'f_agg': 'min'},
  {'attr': 'rvalue', 'chunk_len': 1000, 'f_agg': 'mean'},
  {'attr': 'rvalue', 'chunk_len': 1000, 'f_agg': 'var'},
  {'attr': 'rvalue', 'chunk_len': 5000, 'f_agg': 'max'},
  {'attr': 'rvalue', 'chunk_len': 5000, 'f_agg': 'min'},
  {'attr': 'rvalue', 'chunk_len': 5000, 'f_agg': 'mean'},
  {'attr': 'rvalue', 'chunk_len': 5000, 'f_agg': 'var'},
  {'attr': 'intercept', 'chunk_len': 500, 'f_agg': 'max'},
  {'attr': 'intercept', 'chunk_len': 500, 'f_agg': 'min'},
  {'attr': 'intercept', 'chunk_len': 500, 'f_agg': 'mean'},
  {'attr': 'intercept', 'chunk_len': 500, 'f_agg': 'var'},
  {'attr': 'intercept', 'chunk_len': 1000, 'f_agg': 'max'},
  {'attr': 'intercept', 'chunk_len': 1000, 'f_agg': 'min'},
  {'attr': 'intercept', 'chunk_len': 1000, 'f_agg': 'mean'},
  {'attr': 'intercept', 'chunk_len': 1000, 'f_agg': 'var'},
  {'attr': 'intercept', 'chunk_len': 5000, 'f_agg': 'max'},
  {'attr': 'intercept', 'chunk_len': 5000, 'f_agg': 'min'},
  {'attr': 'intercept', 'chunk_len': 5000, 'f_agg': 'mean'},
  {'attr': 'intercept', 'chunk_len': 5000, 'f_agg': 'var'},
  {'attr': 'slope', 'chunk_len': 500, 'f_agg': 'max'},
  {'attr': 'slope', 'chunk_len': 500, 'f_agg': 'min'},
  {'attr': 'slope', 'chunk_len': 500, 'f_agg': 'mean'},
  {'attr': 'slope', 'chunk_len': 500, 'f_agg': 'var'},
  {'attr': 'slope', 'chunk_len': 1000, 'f_agg': 'max'},
  {'attr': 'slope', 'chunk_len': 1000, 'f_agg': 'min'},
  {'attr': 'slope', 'chunk_len': 1000, 'f_agg': 'mean'},
  {'attr': 'slope', 'chunk_len': 1000, 'f_agg': 'var'},
  {'attr': 'slope', 'chunk_len': 5000, 'f_agg': 'max'},
  {'attr': 'slope', 'chunk_len': 5000, 'f_agg': 'min'},
  {'attr': 'slope', 'chunk_len': 5000, 'f_agg': 'mean'},
  {'attr': 'slope', 'chunk_len': 5000, 'f_agg': 'var'},
  {'attr': 'stderr', 'chunk_len': 500, 'f_agg': 'max'},
  {'attr': 'stderr', 'chunk_len': 500, 'f_agg': 'min'},
  {'attr': 'stderr', 'chunk_len': 500, 'f_agg': 'mean'},
  {'attr': 'stderr', 'chunk_len': 500, 'f_agg': 'var'},
  {'attr': 'stderr', 'chunk_len': 1000, 'f_agg': 'max'},
  {'attr': 'stderr', 'chunk_len': 1000, 'f_agg': 'min'},
  {'attr': 'stderr', 'chunk_len': 1000, 'f_agg': 'mean'},
  {'attr': 'stderr', 'chunk_len': 1000, 'f_agg': 'var'},
  {'attr': 'stderr', 'chunk_len': 5000, 'f_agg': 'max'},
  {'attr': 'stderr', 'chunk_len': 5000, 'f_agg': 'min'},
  {'attr': 'stderr', 'chunk_len': 5000, 'f_agg': 'mean'},
  {'attr': 'stderr', 'chunk_len': 5000, 'f_agg': 'var'}],
 'index_mass_quantile': [{'q': 0.1},
  {'q': 0.2},
  {'q': 0.3},
  {'q': 0.4},
  {'q': 0.6},
  {'q': 0.7},
  {'q': 0.8},
  {'q': 0.9}],

 'spkt_welch_density': [{'coeff': 2}, {'coeff': 5}, {'coeff': 8}],
 'ar_coefficient': [{'coeff': 0, 'k': 10},
  {'coeff': 1, 'k': 10},
  {'coeff': 2, 'k': 10},
  {'coeff': 3, 'k': 10},
  {'coeff': 4, 'k': 10}],

 'energy_ratio_by_chunks': [{'num_segments': 10, 'segment_focus': 0},
  {'num_segments': 10, 'segment_focus': 1},
  {'num_segments': 10, 'segment_focus': 2},
  {'num_segments': 10, 'segment_focus': 3},
  {'num_segments': 10, 'segment_focus': 4},
  {'num_segments': 10, 'segment_focus': 5},
  {'num_segments': 10, 'segment_focus': 6},
  {'num_segments': 10, 'segment_focus': 7},
  {'num_segments': 10, 'segment_focus': 8},
  {'num_segments': 10, 'segment_focus': 9}],
 'ratio_beyond_r_sigma': [{'r': 0.5},
  {'r': 1},
  {'r': 1.5},
  {'r': 2},
  {'r': 2.5},
  {'r': 3},
  {'r': 5},
  {'r': 6},
  {'r': 7},
  {'r': 10}],
'max_langevin_fixed_point': [{'m': 3, 'r': 30}],

}
peak={
'number_peaks': [{'n': 1}, 
                  {'n': 3}, 
                  {'n': 5}, 
                  {'n': 10}]}

## Comp. Analysis

In [0]:
train_t = train[:150000]
train_t = pd.concat([pd.DataFrame({'id': np.ones(train_t.shape[0])}, dtype=np.int16),
                     pd.DataFrame({'time': range(train_t.shape[0])}, dtype=np.int32),
                     train_t.iloc[:,0]],axis=1)

print(train_t.shape)
print(train_t.head())

(4194, 508)
   id  time  acoustic_data__number_peaks__n_1
0   1     0                               NaN
1   1     1                               NaN
2   1     2                               NaN
3   1     3                               NaN
4   1     4                               NaN


In [0]:
res  = pd.DataFrame()
n_ts = 20
l_ts = 1000
n_ti = 3

def tsfresh_time(sett):
  for f, param in tqdm_notebook(sett.items()):
    res.loc[f, "feature"] = f
    res.loc[f, "n_samp"] = n_ts
    res.loc[f, "length"] = l_ts

    fc_dict = {f:param}

    t = timeit.timeit(lambda : extract_features(train_t, 
                     column_id='id',
                     column_sort='time',
                     column_value="acoustic_data",
                     n_jobs=1, 
                     default_fc_parameters=fc_dict, 
                     disable_progressbar=True), 
                      number=n_ti)
    n_fs = 1
    res.loc[f, "n_fs"] = n_fs
    res.loc[f, "t_abs"] = t * 1.0/n_fs
    res.loc[f, "t_1ts"] = t*1.0/(n_ts*n_fs)
  return res

In [0]:
res = pd.DataFrame()
res = tsfresh_time(settings)
res["feature"] = res.feature.astype(str)
res = res.sort_values(by="feature")

plt.figure(figsize=(6, 20))
sns.barplot(y="feature", x="t_abs", data=res)
plt.title("Runtime of 1 apply features for 1 time series of length 15000")
plt.show()

## Create derived features

In [0]:
def ts_features(train1,settings):
  X = extract_features(train1, column_id='id', column_sort='time', 
                       column_value="acoustic_data", 
                       default_fc_parameters=settings,
                       n_jobs = 0,
#                        impute_function = impute,
                       disable_progressbar=True,
                       show_warnings=True)
  return X

In [0]:
train_path = '/content/train.csv'
n = 150000

parts_n = 4
parts_seg = n//parts_n

len_df = 629145480  # sum(1 for line in open(train_path))-1
segments = ((len_df + n - 1)//n)-1

ids = pd.DataFrame({'id': np.ones(n)}, dtype=np.int16)
times = pd.DataFrame({'time': range(n)}, dtype=np.int32)

y_tr = pd.DataFrame(index=range(segments), dtype=np.float64,
                    columns=['time_to_failure'])

x_processed = pd.DataFrame()
for chunk in tqdm_notebook(pd.read_csv(train_path, chunksize=n),
                           total=segments):
  
  p_placehold = pd.DataFrame()
  
  train_placehold = pd.concat([ids,
                               times,
                               chunk.reset_index(drop=True)],
                              axis=1)
  
  y = chunk['time_to_failure'].values[-1]  # TODO explore
  y_tr.loc[i, 'time_to_failure'] = y
  
  x_features = ts_features(train_placehold, y)  # Features in whole segment
  
  for j in range(parts_n):  # Features in quarters
      
      p_features = ts_features(train_placehold[j*parts_seg : (j+1)*parts_seg],
                               settings).add_prefix('q_'+str(j))
      
      p_placehold = pd.concat([p_placehold, p_features], axis = 1)
      
  x_processed = x_processed.append(pd.concat([x_features, p_placehold], 
                                             axis=1))
#   print(x_processed.head())

In [0]:
train_path = '/content/train.csv'
n = 150000

parts_n = 4
parts_seg = n//parts_n

len_df = 629145480  # sum(1 for line in open(train_path))-1
segments = ((len_df + n - 1)//n)-1

ids = pd.DataFrame({'id': np.ones(n)}, dtype=np.int16)
times = pd.DataFrame({'time': range(n)}, dtype=np.int32)


for chunk in tqdm_notebook(pd.read_csv(train_path, chunksize=n),
                           total=segments):
  

  train_placehold = pd.concat([ids,
                               times,
                               chunk.reset_index(drop=True)],
                              axis=1)
  
  x = train_placehold[['id','time','acoustic_data']]
  
  y = train_placehold['time_to_failure']
  y.index = np.ones(n)
  
  x_feat = extract_relevant_features(x, y, column_id='id',
                                     column_sort='time', 
                                     column_value='acoustic_data', 
                                     default_fc_parameters=EfficientFCParameters(),
                                     n_jobs = 0,
#                                      chunksize = 0, 
                                     disable_progressbar=True,
                                     show_warnings=True)
  break


In [0]:
y.head()

In [0]:
chunk['time_to_failure'].values[-1]

9.7597955148

In [0]:
X_filtered = x_processed.reset_index().drop(['id'], axis=1)

print(X_filtered.shape)
print(y_tr.shape)
# del train

### Feature Extraction for submission

In [0]:
submission = pd.read_csv('sample_submission.csv', index_col='seg_id')

X_testfresh = pd.DataFrame()
# Load each test data, create the feature matrix, get numeric prediction
for i in tqdm_notebook(range(len(submission))): 
#   print(i)
  X1 = pd.DataFrame()
  ids = pd.DataFrame({'id': [submission.index[i] for number in range(n)]}, dtype=np.int16)
  seg = pd.read_csv(submission.index[i] + '.csv' )
  x = seg['acoustic_data']
  test=pd.concat([ids,times,x],axis=1)

#   Features in whole segment
  X2=ts_features(test,peak)
#   Features per section 5x30000
  for j in range(4):
      X=ts_features(test[j * parts:(j + 1) *parts],settings).add_prefix('q_' +str(j))
      X1=pd.concat([X1,X], axis=1)
      
#   print(X1.shape)
  X_testfresh = X_testfresh.append(pd.concat([X1,X2], axis=1))
  

## Save/Load Data

In [0]:
# X_filtered.to_csv('X_filtered.csv', index=False)
# X_testfresh.to_csv('X_testfresh.csv', index=False)
# y_tr.to_csv('y_tr.csv', index=False)

X_filtered.head()

In [0]:
# X_filtered_loaded = pd.read_csv('X_filtered.csv')
X_testfresh_loaded = pd.read_csv('X_testfresh.csv', index_col='id')
# y_tr_loaded = pd.read_csv('y_tr.csv')

# print(X_filtered_loaded.shape)
print(X_testfresh_loaded.shape)
# print(y_tr_loaded.shape)

(2624, 520)


In [0]:
# X_filteredd = X_filtered_loaded 
X_testfresh = X_testfresh_loaded
# y_tr = y_tr_loaded

In [0]:
X_testfresh.head()

### Filtering Data to remove repeating values as well as N/A values.

In [0]:
dfb=X_filtered.reset_index(drop=True).append(X_testfresh)
dfb.to_csv('unfiltered.csv',index=False)
print(dfb.shape)
dfb=dfb.dropna(axis='rows')
dfb=dfb.loc[:, (dfb != dfb.iloc[0]).any()] 
print(dfb.shape)
dfb.head()

In [0]:
df=pd.concat([y_tr,dfb[:X_filtered.shape[0]]], axis=1)
df.to_csv('tsfreshFeaturestrain.csv',index=False)
test=dfb[-X_testfresh.shape[0]:]
test.to_csv('tsfreshFeaturestest.csv',index=False)

X=df[df.columns.difference(['time_to_failure'])]
y=df['time_to_failure']

#Scaling train and test features at the same time
scaler=MinMaxScaler(feature_range=(0, 1))
scaled=scaler.fit_transform(X.append(test))

train=pd.DataFrame(scaled[:X.shape[0]], columns=X.columns.values)
test=pd.DataFrame(scaled[-test.shape[0]:], columns=X.columns.values)

In [0]:
# train_shuffled = train.sample(frac=1)
trainX, testX, trainy, testy = train_test_split(train, y, test_size=0.01, shuffle=True)

## Regression with GBM and LightGBM

### GBM

In [0]:
# np.any(np.isnan(trainy))
np.all(np.isfinite(trainX))

True

In [0]:
%%time
model = GradientBoostingRegressor(learning_rate=0.009, loss='huber',
                                  max_depth=5,max_features=150,n_estimators=650)
num_folds = 5
cross_validate_results = cross_validate(model, trainX, trainy, n_jobs=-1,
                                        return_estimator= True,
                                        return_train_score=True, cv=num_folds,
                                        scoring="neg_mean_absolute_error" )

for i in range(num_folds):
    print("Fold: {} Training MAE: {}".format(i,-cross_validate_results['train_score'][i]))
    print("Fold: {} Validation MAE: {}".format(i,-cross_validate_results['test_score'][i]))
print(-cross_validate_results['train_score'].mean(), cross_validate_results['train_score'].std())
print(-cross_validate_results['test_score'].mean(), cross_validate_results['test_score'].std())

model.fit(trainX, trainy)
data=pd.concat([pd.DataFrame(X.columns.values,columns={'Features'}),
           pd.DataFrame(model.feature_importances_,columns={'Importance'})*100],
               axis=1).sort_values(by='Importance', ascending=False)

data.to_csv('FeaturesImportance.csv', index=False)

plt.figure(figsize=(6, 20))
sns.barplot(y="Features", x="Importance", data=data[:150])
plt.title("Features")
plt.show()

ValueError: ignored

In [0]:
data.head(20)

In [0]:
from sklearn.metrics import mean_absolute_error
mae = np.array([])
predictions1 = np.zeros(len(test))
feature_importance_df = pd.DataFrame()
#run model

n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)
train_columns = trainX.columns.values

for fold_, (trn_idx, val_idx) in enumerate(folds.split(trainX,trainy.values)):
    strLog = "fold {}".format(fold_)
    
    X_tr, X_val = trainX.iloc[trn_idx], trainX.iloc[val_idx]
    y_tr, y_val = trainy.iloc[trn_idx], trainy.iloc[val_idx]

    
    model.fit(X_tr, y_tr)
    
    mae = np.append(mae, mean_absolute_error(y_val, model.predict(X_val)))
    print("Fold: {} Validation MAE: {}".format(strLog,mae[fold_]))
  
    #predictions
    predictions1 += model.predict(test) / folds.n_splits
np.mean(mae)

ValueError: ignored

In [0]:
submission.time_to_failure = predictions1
submission.to_csv('submissiongbm.csv',index=True)
submission.head()

,time_to_failure
seg_id,
seg_00030f,3.698928
seg_0012b5,6.098628
seg_00184e,6.680970
seg_003339,8.088127
seg_0042cc,7.376599


### LightGBM

In [0]:
params={'bagging_fraction': 0.72,
        'boosting': 'gbdt',
        'feature_fraction' : 0.55,
        'lambda_l1': 4.2030167201177235,
        'lambda_l2': 2.8818379730100174,
        'learning_rate': 0.04197920600013021,
        'max_bin': 182,
        'max_depth': 13,
        'metric': 'RMSE',
        'min_data_in_bin': 106,
        'min_data_in_leaf': 83,
        'num_leaves': 1415,
        'objective': 'fair',
        'subsample': 0.5807414994404817}
predictions2 = np.zeros(len(test))
feature_importance_df = pd.DataFrame()
#run model

n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)
train_columns = trainX.columns.values

for fold_, (trn_idx, val_idx) in enumerate(folds.split(trainX,trainy.values)):
    strLog = "fold {}".format(fold_)
    
    X_tr, X_val = trainX.iloc[trn_idx], trainX.iloc[val_idx]
    y_tr, y_val = trainy.iloc[trn_idx], trainy.iloc[val_idx]

    print(strLog)
    
    model = lgb.LGBMRegressor(**params, n_estimators = 20000, n_jobs = -1)
    model.fit(X_tr, 
              y_tr, 
              eval_set=[(X_tr, y_tr), (X_val, y_val)], 
              eval_metric='mae',
              verbose=1000, 
              early_stopping_rounds=1000)
    mae = np.append(mae, mean_absolute_error(y_val, model.predict(X_val)))
    print("Fold: {} Validation MAE: {}".format(strLog,mae[fold_]))
    #predictions
    predictions2 += model.predict(test, num_iteration=model.best_iteration_) / folds.n_splits
np.mean(mae)

fold 0
Training until validation scores don't improve for 1000 rounds.
[1000]	training's l1: nan	training's rmse: nan	valid_1's l1: nan	valid_1's rmse: nan
Early stopping, best iteration is:
[1]	training's l1: nan	training's rmse: nan	valid_1's l1: nan	valid_1's rmse: nan


NameError: ignored

# LANL Earthquake with H2O AutoML

In [0]:
!pip install h2o

     |████████████████████████████████| 121.8MB 161kB/s 
  Stored in directory: /root/.cache/pip/wheels/36/73/57/2ea67a8a35ea9114f7d66cc8c5ec46c915698c909321f8a17d
Successfully built h2o


In [0]:
import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init() # max_mem_size='16G'

3.24.0.3
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.3" 2019-04-16; OpenJDK Runtime Environment (build 11.0.3+7-Ubuntu-1ubuntu218.04.1); OpenJDK 64-Bit Server VM (build 11.0.3+7-Ubuntu-1ubuntu218.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpqpcn64ay
  JVM stdout: /tmp/tmpqpcn64ay/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpqpcn64ay/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.3
H2O cluster version age:,"14 days, 5 hours and 19 minutes"
H2O cluster name:,H2O_from_python_unknownUser_a6wgtc
H2O cluster total nodes:,1
H2O cluster free memory:,2.938 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [0]:
train = h2o.import_file("X_tr.csv")
test = h2o.import_file("X_test.csv")
y_train = h2o.import_file("y_tr.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
train.head()

mean,std,max,min,mean_change_abs,mean_change_rate,abs_max,abs_min,std_first_50000,std_last_50000,std_first_10000,std_last_10000,avg_first_50000,avg_last_50000,avg_first_10000,avg_last_10000,min_first_50000,min_last_50000,min_first_10000,min_last_10000,max_first_50000,max_last_50000,max_first_10000,max_last_10000,max_to_min,max_to_min_diff,count_big,sum,mean_change_rate_first_50000,mean_change_rate_last_50000,mean_change_rate_first_10000,mean_change_rate_last_10000,q95,q99,q05,q01,abs_q95,abs_q99,abs_q05,abs_q01,trend,abs_trend,abs_mean,abs_std,mad,kurt,skew,med,Hilbert_mean,Hann_window_mean,classic_sta_lta1_mean,classic_sta_lta2_mean,classic_sta_lta3_mean,classic_sta_lta4_mean,Moving_average_700_mean,Moving_average_1500_mean,Moving_average_3000_mean,Moving_average_6000_mean,exp_Moving_average_300_mean,exp_Moving_average_3000_mean,exp_Moving_average_30000_mean,MA_700MA_std_mean,MA_700MA_BB_high_mean,MA_700MA_BB_low_mean,MA_400MA_std_mean,MA_400MA_BB_high_mean,MA_400MA_BB_low_mean,MA_1000MA_std_mean,iqr,q999,q001,ave10,ave_roll_std_10,std_roll_std_10,max_roll_std_10,min_roll_std_10,q01_roll_std_10,q05_roll_std_10,q95_roll_std_10,q99_roll_std_10,av_change_abs_roll_std_10,av_change_rate_roll_std_10,abs_max_roll_std_10,ave_roll_mean_10,std_roll_mean_10,max_roll_mean_10,min_roll_mean_10,q01_roll_mean_10,q05_roll_mean_10,q95_roll_mean_10,q99_roll_mean_10,av_change_abs_roll_mean_10,av_change_rate_roll_mean_10,abs_max_roll_mean_10,ave_roll_std_100,std_roll_std_100,max_roll_std_100,min_roll_std_100,q01_roll_std_100,q05_roll_std_100,q95_roll_std_100,q99_roll_std_100,av_change_abs_roll_std_100,av_change_rate_roll_std_100,abs_max_roll_std_100,ave_roll_mean_100,std_roll_mean_100,max_roll_mean_100,min_roll_mean_100,q01_roll_mean_100,q05_roll_mean_100,q95_roll_mean_100,q99_roll_mean_100,av_change_abs_roll_mean_100,av_change_rate_roll_mean_100,abs_max_roll_mean_100,ave_roll_std_1000,std_roll_std_1000,max_roll_std_1000,min_roll_std_1000,q01_roll_std_1000,q05_roll_std_1000,q95_roll_std_1000,q99_roll_std_1000,av_change_abs_roll_std_1000,av_change_rate_roll_std_1000,abs_max_roll_std_1000,ave_roll_mean_1000,std_roll_mean_1000,max_roll_mean_1000,min_roll_mean_1000,q01_roll_mean_1000,q05_roll_mean_1000,q95_roll_mean_1000,q99_roll_mean_1000,av_change_abs_roll_mean_1000,av_change_rate_roll_mean_1000,abs_max_roll_mean_1000
4.88411,5.10111,104,-98,-8.00005e-05,74836.6,104,0,6.48855,3.66466,11.2072,4.36141,4.9621,4.62046,5.182,4.6756,-98,-21,-98,-16,104,30,104,26,1.06122,6,0,732617,24942.3,25002.3,4981.15,4983.58,11,18,-2,-8,12,20,1,0,-3.2683e-06,-1.12785e-05,5.57657,4.33332,3.2634,33.6625,-0.0240612,5,7.02703,4.88333,0.920976,0.266087,0.935911,0.808124,4.88359,4.88267,4.8805,4.87777,4.88465,4.88735,4.89181,4.22942,13.3424,-3.57524,4.15581,13.1952,-3.42802,4.28859,4,40,-30,4.86934,3.50712,2.80907,78.313,0.421637,1.26491,1.63639,7.22649,15.0342,-1.53422e-06,74731,78.313,4.88409,2.8018,68.5,-60,-2.4,1.7,8.1,12.4,-1.13341e-05,74736.7,68.5,4.05045,3.11152,52.3353,1.93542,2.30263,2.47564,8.1959,16.9488,-5.23545e-06,74759.6,52.3353,4.88386,0.452294,10.04,1.04,3.81,4.16,5.58,5.95,-2.4016e-06,74760.1,10.04,4.28859,2.76977,31.0294,2.53663,2.61609,2.70647,8.18576,15.056,-1.17955e-05,74223.6,31.0294,4.88342,0.295715,5.629,3.896,4.072,4.379,5.338,5.484,-1.7047e-06,74222.3,5.629
4.72577,6.58882,181,-154,0,74891.7,181,0,7.30523,5.49307,3.97675,3.66789,4.6984,4.78834,4.7772,4.9144,-154,-115,-23,-24,181,111,29,29,1.17532,27,0,708865,25027.3,24957.8,4990.56,4978.31,12,21,-2,-11,12,24,1,0,9.09042e-07,-5.38941e-06,5.73417,5.73278,3.5743,98.7585,0.390561,5,7.38038,4.72505,0.965946,0.285984,0.948883,0.79155,4.72482,4.72491,4.72367,4.7182,4.72605,4.72803,4.72762,4.73535,14.1955,-4.74589,4.60858,13.942,-4.49234,4.84349,5,59,-47,4.7242,3.76143,4.12078,122.978,0.421637,1.28668,1.64655,8.26573,18.7727,-3.28446e-06,74835.4,122.978,4.72573,3.92407,145.1,-128,-4.6,1.2,8.2,13.8,2.66684e-06,74828.6,145.1,4.43636,4.89343,87.9726,1.99137,2.30029,2.47597,9.82992,23.4573,-4.87196e-07,74802

In [0]:
train['time_to_failure'] = y_train['time_to_failure']

In [0]:
%%time
x = test.columns
y = 'time_to_failure'

aml = H2OAutoML(max_models=100, max_runtime_secs=3600, stopping_metric='MAE') # seed=33,
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%
CPU times: user 10.3 s, sys: 1.08 s, total: 11.4 s
Wall time: 46min 14s


In [0]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_grid_1_AutoML_20190521_101044_model_9,7.04533,2.6543,7.04533,2.0712,0.510846
GBM_grid_1_AutoML_20190521_084240_model_29,7.06034,2.65713,7.06034,2.0784,0.512125
GBM_grid_1_AutoML_20190521_101044_model_12,7.06723,2.65843,7.06723,2.07569,0.5129
GBM_grid_1_AutoML_20190521_084240_model_53,7.0928,2.66323,7.0928,2.06949,0.510405
GBM_grid_1_AutoML_20190521_084240_model_14,7.09696,2.66401,7.09696,2.0739,0.510519
GBM_5_AutoML_20190521_101044,7.1001,2.6646,7.1001,2.07294,0.510387
GBM_grid_1_AutoML_20190521_084240_model_17,7.11195,2.66682,7.11195,2.07862,0.511922
XGBoost_grid_1_AutoML_20190521_084240_model_8,7.12039,2.66841,7.12039,2.06507,0.507323
GBM_5_AutoML_20190521_084240,7.12392,2.66907,7.12392,2.07954,0.511664
XGBoost_3_AutoML_20190521_084240,7.1484,2.67365,7.1484,2.06226,0.506703


In [0]:
# The leader model is stored here
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_1_AutoML_20190521_101044_model_9


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 5.839327202282622
RMSE: 2.4164699878712796
MAE: 1.8806869073605061
RMSLE: 0.47686789177031974
Mean Residual Deviance: 5.839327202282622

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 7.0453328853967845
RMSE: 2.6543045954443105
MAE: 2.0711995847559312
RMSLE: 0.5108457084454961
Mean Residual Deviance: 7.0453328853967845
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,2.071201,0.0140208,2.0364754,2.0638592,2.084558,2.0931556,2.0779574
mean_residual_deviance,7.0453534,0.0635008,6.8727474,7.075518,7.0904613,7.0562754,7.131765
mse,7.0453534,0.0635008,6.8727474,7.075518,7.0904613,7.0562754,7.131765
r2,0.47767,0.0050291,0.4910830,0.4762091,0.4715489,0.4776193,0.4718898
residual_deviance,7.0453534,0.0635008,6.8727474,7.075518,7.0904613,7.0562754,7.131765
rmse,2.6542542,0.0120087,2.6215925,2.6599846,2.662792,2.6563652,2.6705365
rmsle,0.5107755,0.0061049,0.4998769,0.5164882,0.514052,0.5015141,0.5219459


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-05-21 10:27:10,4 min 22.880 sec,0.0,3.6727069,3.0480839,13.4887759
,2019-05-21 10:27:10,4 min 23.050 sec,5.0,3.2817193,2.6857614,10.7696817
,2019-05-21 10:27:10,4 min 23.228 sec,10.0,3.0155625,2.4378020,9.0936172
,2019-05-21 10:27:10,4 min 23.397 sec,15.0,2.8380359,2.2718323,8.0544478
,2019-05-21 10:27:11,4 min 23.571 sec,20.0,2.7194687,2.1637489,7.3955102
,2019-05-21 10:27:11,4 min 23.767 sec,25.0,2.6370053,2.0884801,6.9537968
,2019-05-21 10:27:11,4 min 23.944 sec,30.0,2.5789197,2.0335114,6.6508268
,2019-05-21 10:27:11,4 min 24.131 sec,35.0,2.5372449,1.9943987,6.4376115
,2019-05-21 10:27:11,4 min 24.321 sec,40.0,2.5030146,1.9626128,6.2650823
,2019-05-21 10:27:12,4 min 24.502 sec,45.0,2.4771044,1.9370368,6.1360464


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
q05_roll_std_100,144172.2656250,1.0,0.4499418
q05_roll_std_10,82245.4453125,0.5704665,0.2566767
q05_roll_std_1000,19573.0585938,0.1357616,0.0610848
q01_roll_std_10,10541.9208984,0.0731203,0.0328999
MA_400MA_BB_high_mean,6694.9091797,0.0464369,0.0208939
---,---,---,---
q01,6.7769408,0.0000470,0.0000211
min,0.0,0.0,0.0
count_big,0.0,0.0,0.0
abs_q05,0.0,0.0,0.0



See the whole table with table.as_data_frame()


In [0]:
preds = aml.predict(test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [0]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['time_to_failure'] = preds.as_data_frame().values.flatten()
sample_submission.to_csv('submission.csv', index=False)

In [0]:
!kaggle competitions submit -c LANL-Earthquake-Prediction -f submission.csv -m "h2o_submission2" 

100% 74.8k/74.8k [00:03<00:00, 20.6kB/s]
Successfully submitted to LANL Earthquake Prediction